## 1. Create Llama Stack client, list available models and vector databases

In [37]:
from llama_stack_client import LlamaStackClient
client = LlamaStackClient(base_url="http://lsd-llama-milvus-service:8321")

models = client.models.list()
print(f"Models information: {models}\n")

inference_llm = next((model.identifier for model in models if model.model_type == 'llm'), None)
print(f"Identifier for Inference model in usage: {inference_llm}\n")

# Check what vector databases exist
print("=== Available Vector Databases ===")
vector_dbs = client.vector_dbs.list()
if vector_dbs:
    for vdb in vector_dbs:
        print(f"- ID: {vdb.identifier}")
        print(f"  Provider: {vdb.provider_id}")
        print(f"  Embedding Model: {vdb.embedding_model}")
        print()
else:
    print("No vector databases found!")

INFO:httpx:HTTP Request: GET http://lsd-llama-milvus-service:8321/v1/models "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://lsd-llama-milvus-service:8321/v1/vector-dbs "HTTP/1.1 200 OK"


Models information: [Model(identifier='vllm', metadata={}, api_model_type='llm', provider_id='vllm-inference', type='model', provider_resource_id='vllm', model_type='llm'), Model(identifier='granite-embedding-125m', metadata={'embedding_dimension': 768.0}, api_model_type='embedding', provider_id='sentence-transformers', type='model', provider_resource_id='ibm-granite/granite-embedding-125m-english', model_type='embedding')]

Identifier for Inference model in usage: vllm

=== Available Vector Databases ===
- ID: csv-vector-db
  Provider: milvus
  Embedding Model: granite-embedding-125m



## 2. Create RAG Agent and prompt the LLM
Prompt the LLM with questions in relation to the documents inserted, and see it return accurate answers.

In [38]:
from llama_stack_client import Agent, AgentEventLogger
import uuid

rag_agent = Agent(
    client,
    model="vllm",
    instructions="You are a helpful assistant. Answer the user's question based only on the provided search results. Respond with 'I don’t know' if the information is outside of the scope of your knowledge and not present in the search results.",
    tools=[
        {
            "name": "builtin::rag/knowledge_search",
            "args": {"vector_db_ids": ["csv-vector-db"]},
        }
    ],
)

user_prompts = [
    "What is gender, country and age of Dulce Abril and Philip Gent?",
    "What is customer id, company, city, country, phone number, email, subscription date, subscribed website of of Sheryl Baxter?",
    "List me the revenue from product Widget D and quantity sold on 20/01/2024",
    "What is the economics condition at Ireland in 2025?", # Dummy question the model will answer with 'I don’t know' 
]

session_id = rag_agent.create_session(session_name=f"s{uuid.uuid4().hex}")

for prompt in user_prompts:
    print("prompt>", prompt)
    response = rag_agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=True,
    )
    for log in AgentEventLogger().log(response):
        log.print()

# Get session response for further evaluation of RAG metrics
session_response = client.agents.session.retrieve(
    session_id=session_id,
    agent_id=rag_agent.agent_id,
)

INFO:httpx:HTTP Request: POST http://lsd-llama-milvus-service:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://lsd-llama-milvus-service:8321/v1/tools?toolgroup_id=builtin%3A%3Arag%2Fknowledge_search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://lsd-llama-milvus-service:8321/v1/agents/ab396f74-efc7-4503-bacd-9c3361fe6243/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://lsd-llama-milvus-service:8321/v1/agents/ab396f74-efc7-4503-bacd-9c3361fe6243/session/775fcedc-81b7-4360-be76-7e209f301365/turn "HTTP/1.1 200 OK"


prompt> What is gender, country and age of Dulce Abril and Philip Gent?
inference> 
tool_execution> Tool:knowledge_search Args:{'query': 'Dulce Abril and Philip Gent gender, country, age'}
tool_execution> Tool:knowledge_search Response:[TextContentItem(text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n', type='text'), TextContentItem(text="Result 1\nContent: 1, First Name = Dulce. 1, Last Name = Abril. 1, Gender = Female. 1, Country = United States. 1, Age = 32. 1, Date = 15/10/2017. 1, Id = 1562. 2, First Name = Mara. 2, Last Name = Hashimoto. 2, Gender = Female. 2, Country = Great Britain. 2, Age = 25. 2, Date = 16/08/2016. 2, Id = 1582. 3, First Name = Philip. 3, Last Name = Gent. 3, Gender = Male. 3, Country = France. 3, Age = 36. 3, Date = 21/05/2015. 3, Id = 2587. 4, First Name = Kathleen. 4, Last Name = Hanner. 4, Gender = Female. 4, Country = United States. 4, Age = 25. 4, Date = 15/10/2017. 4, Id = 3549. 5, First Name = Nereida. 5, Last Name

INFO:httpx:HTTP Request: POST http://lsd-llama-milvus-service:8321/v1/agents/ab396f74-efc7-4503-bacd-9c3361fe6243/session/775fcedc-81b7-4360-be76-7e209f301365/turn "HTTP/1.1 200 OK"


prompt> What is customer id, company, city, country, phone number, email, subscription date, subscribed website of of Sheryl Baxter?
inference> 
tool_execution> Tool:knowledge_search Args:{'query': 'Sheryl Baxter customer id, company, city, country, phone number, email, subscription date, subscribed website'}
tool_execution> Tool:knowledge_search Response:[TextContentItem(text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n', type='text'), TextContentItem(text="Result 1\nContent: 1, Customer Id = DD37Cf93aecA6Dc. 1, First Name = Sheryl. 1, Last Name = Baxter. 1, Company = Rasmussen Group. 1, City = East Leonard. 1, Country = Chile. 1, Phone 1 = 229.077.5154. 1, Phone 2 = 397.884.0519x718. 1, Email = zunigavanessa@smith.info. 1, Subscription Date = 2020-08-24. 1, Website = http://www.stephenson.com/. 2, Customer Id = 1Ef7b82A4CAAD10. 2, First Name = Preston. 2, Last Name = Lozano, Dr. 2, Company = Vega-Gentry. 2, City = East Jimmychester. 2, Country = D

INFO:httpx:HTTP Request: POST http://lsd-llama-milvus-service:8321/v1/agents/ab396f74-efc7-4503-bacd-9c3361fe6243/session/775fcedc-81b7-4360-be76-7e209f301365/turn "HTTP/1.1 200 OK"


prompt> List me the revenue from product Widget D and quantity sold on 20/01/2024
inference> 
tool_execution> Tool:knowledge_search Args:{'query': 'revenue from product Widget D and quantity sold on 20/01/2024'}
tool_execution> Tool:knowledge_search Response:[TextContentItem(text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n', type='text'), TextContentItem(text="Result 1\nContent: 12500. Widget C, Date = 2024-01-19. Widget C, Quantity = 6. Widget C, Revenue = 6100. Widget D, Date = 2024-01-20. Widget D, Quantity = 8. Widget D, Revenue = 8900\nMetadata: {'file_name': 'sample_sales_data_SalesData', 'document_id': 'b162f936-3068-4d7e-bca1-a5b5d14fa11c'}\n", type='text'), TextContentItem(text="Result 2\nContent: 12500. Widget C, Date = 2024-01-19. Widget C, Quantity = 6. Widget C, Revenue = 6100. Widget D, Date = 2024-01-20. Widget D, Quantity = 8. Widget D, Revenue = 8900\nMetadata: {'file_name': 'sample_sales_data_SalesData', 'document_id': '62c91f75-1

INFO:httpx:HTTP Request: POST http://lsd-llama-milvus-service:8321/v1/agents/ab396f74-efc7-4503-bacd-9c3361fe6243/session/775fcedc-81b7-4360-be76-7e209f301365/turn "HTTP/1.1 200 OK"


prompt> What is the economics condition at Ireland in 2025?
inference> 
tool_execution> Tool:knowledge_search Args:{'query': 'economics condition in Ireland in 2025'}
tool_execution> Tool:knowledge_search Response:[TextContentItem(text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n', type='text'), TextContentItem(text="Result 1\nContent: 12500. Widget C, Date = 2024-01-19. Widget C, Quantity = 6. Widget C, Revenue = 6100. Widget D, Date = 2024-01-20. Widget D, Quantity = 8. Widget D, Revenue = 8900\nMetadata: {'file_name': 'sample_sales_data_SalesData', 'document_id': '62c91f75-10ee-4300-b99c-78bb96d5e023'}\n", type='text'), TextContentItem(text="Result 2\nContent: 12500. Widget C, Date = 2024-01-19. Widget C, Quantity = 6. Widget C, Revenue = 6100. Widget D, Date = 2024-01-20. Widget D, Quantity = 8. Widget D, Revenue = 8900\nMetadata: {'file_name': 'sample_sales_data_SalesData', 'document_id': 'b162f936-3068-4d7e-bca1-a5b5d14fa11c'}\n", type='text')

INFO:httpx:HTTP Request: GET http://lsd-llama-milvus-service:8321/v1/agents/ab396f74-efc7-4503-bacd-9c3361fe6243/session/775fcedc-81b7-4360-be76-7e209f301365 "HTTP/1.1 200 OK"
